# Memory-Efficient Sign Language Detection

This notebook demonstrates the memory-efficient implementation of sign language detection using the modified WLASL framework. It includes:

1. Memory-Efficient Data Preprocessing
2. Optimized Data Loading
3. Memory-Aware Training
4. Performance Monitoring

In [ ]:
import sys
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import psutil
import gc

# Add project root to path
if str(Path().absolute().parent) not in sys.path:
    sys.path.append(str(Path().absolute().parent))

from src.config import *
from src.data.preprocessing import MemoryEfficientPreprocessor
from src.data.loader import (
    MemoryEfficientDataset,
    create_data_loaders,
    StreamingDataLoader
)
from src.training.trainer import MemoryEfficientTrainer

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Enable interactive plots
%matplotlib inline
plt.style.use('seaborn')

## 1. Memory Usage Monitoring

First, let's define some utility functions to monitor memory usage throughout our pipeline.

In [ ]:
def get_memory_usage():
    """Get current memory usage statistics."""
    process = psutil.Process()
    memory_info = process.memory_info()
    
    stats = {
        'RAM Used (GB)': memory_info.rss / 1024**3,
        'RAM Total (GB)': psutil.virtual_memory().total / 1024**3,
        'RAM Usage %': process.memory_percent()
    }
    
    if torch.cuda.is_available():
        stats.update({
            'GPU Memory Used (GB)': torch.cuda.memory_allocated() / 1024**3,
            'GPU Memory Reserved (GB)': torch.cuda.memory_reserved() / 1024**3
        })
    
    return stats

def display_memory_usage():
    """Display current memory usage."""
    stats = get_memory_usage()
    for key, value in stats.items():
        print(f"{key}: {value:.2f}")

# Initial memory usage
print("Initial Memory Usage:")
display_memory_usage()

## 2. Memory-Efficient Video Preprocessing

Now let's preprocess our video data using chunk-based processing to manage memory usage.

In [ ]:
# Initialize preprocessor
preprocessor = MemoryEfficientPreprocessor(
    output_dir=PROCESSED_DIR / 'frames',
    frame_size=PREPROCESSING_CONFIG['frame_size'],
    target_fps=PREPROCESSING_CONFIG['target_fps'],
    chunk_size=PREPROCESSING_CONFIG['chunk_size'],
    max_frames=PREPROCESSING_CONFIG['max_frames'],
    num_workers=PREPROCESSING_CONFIG['num_workers']
)

# Get video paths
video_dir = Path('video')
video_paths = list(video_dir.glob('**/*.mp4'))
print(f"Found {len(video_paths)} videos")

# Process videos with memory monitoring
memory_stats = []
results = []

for i, video_path in enumerate(video_paths):
    try:
        result = preprocessor.preprocess_video(video_path)
        results.append(result)
        
        # Monitor memory every 10 videos
        if i % 10 == 0:
            memory_stats.append(get_memory_usage())
            print(f"\nProcessed {i+1}/{len(video_paths)} videos")
            display_memory_usage()
            
            # Cleanup if needed
            if memory_stats[-1]['RAM Usage %'] > MEMORY_THRESHOLDS['cpu_warning'] * 100:
                gc.collect()
                
    except Exception as e:
        print(f"Error processing {video_path}: {str(e)}")

# Save preprocessing results
with open(PROCESSED_DIR / 'preprocessing_results.json', 'w') as f:
    json.dump(results, f, indent=2)

# Plot memory usage during preprocessing
plt.figure(figsize=(12, 6))
ram_usage = [stats['RAM Usage %'] for stats in memory_stats]
plt.plot(ram_usage, label='RAM Usage %')
plt.title('Memory Usage During Preprocessing')
plt.xlabel('Processing Steps (per 10 videos)')
plt.ylabel('Memory Usage (%)')
plt.legend()
plt.show()

## 3. Memory-Efficient Data Loading

Let's set up our data loading pipeline with efficient memory management.

In [ ]:
# Load preprocessing results
with open(PROCESSED_DIR / 'preprocessing_results.json', 'r') as f:
    data_info = json.load(f)

# Create memory-efficient data loaders
dataloaders = create_data_loaders(
    data_info=data_info,
    processed_dir=PROCESSED_DIR / 'frames',
    batch_size=DATA_CONFIG['batch_size'],
    num_workers=DATA_CONFIG['num_workers'],
    frame_cache_size=DATA_CONFIG['frame_cache_size']
)

print("\nDataset sizes:")
for split, loader in dataloaders.items():
    print(f"{split}: {len(loader.dataset)} samples")

# Test data loading with memory monitoring
print("\nTesting data loading...")
for i, (frames, labels) in enumerate(dataloaders['train']):
    if i == 0:
        print(f"Batch shapes: frames {frames.shape}, labels {labels.shape}")
    
    if i % 10 == 0:
        print(f"\nBatch {i+1}:")
        display_memory_usage()
    
    if i >= 20:  # Test with a few batches
        break

# Clear test data
del frames, labels
gc.collect()
torch.cuda.empty_cache()

## 4. Memory-Efficient Model Training

Now let's train our model using memory-efficient techniques.

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize model (I3D or TGCN)
model_type = 'i3d'  # or 'tgcn'

if model_type == 'i3d':
    from code.I3D.pytorch_i3d import InceptionI3d
    model = InceptionI3d(**I3D_CONFIG)
else:
    from code.TGCN.tgcn_model import TGCN
    model = TGCN(**TGCN_CONFIG)

# Initialize training components
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=TRAIN_CONFIG['learning_rate'],
    weight_decay=TRAIN_CONFIG['weight_decay']
)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=TRAIN_CONFIG['lr_scheduler']['factor'],
    patience=TRAIN_CONFIG['lr_scheduler']['patience'],
    min_lr=TRAIN_CONFIG['lr_scheduler']['min_lr']
)

# Initialize trainer
trainer = MemoryEfficientTrainer(
    model=model,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    config=TRAIN_CONFIG,
    checkpoint_dir=CHECKPOINT_DIR / model_type,
    scheduler=scheduler
)

# Train model
print("\nStarting training...")
history = trainer.train(num_epochs=TRAIN_CONFIG['num_epochs'])

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Validation')
ax1.set_title('Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(history['train_accuracy'], label='Train')
ax2.plot(history['val_accuracy'], label='Validation')
ax2.set_title('Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.tight_layout()
plt.show()

# Display final memory usage
print("\nFinal Memory Usage:")
display_memory_usage()

## 5. Memory Usage Analysis

Let's analyze the memory usage throughout the training process.

In [ ]:
# Load memory statistics
with open(CHECKPOINT_DIR / model_type / 'memory_stats.json', 'r') as f:
    memory_stats = json.load(f)

# Plot memory usage over time
plt.figure(figsize=(12, 6))
epochs = range(len(memory_stats))

if torch.cuda.is_available():
    plt.plot([s['gpu_memory'] for s in memory_stats], label='GPU Memory (MB)')
plt.plot([s['rss'] for s in memory_stats], label='RAM Usage (MB)')

plt.title('Memory Usage During Training')
plt.xlabel('Training Steps')
plt.ylabel('Memory (MB)')
plt.legend()
plt.grid(True)
plt.show()